In [ ]:
from holoviews.streams import PointDraw
from holoviews.plotting.links import DataLink
import holoviews as hv
import panel as pn
#pn.extension() #not sure what this does...
import hvplot.xarray
import hvplot.pandas
from shapely.geometry import LineString
import xarray as xr
import geopandas as gpd
import numpy as np
import os

In [ ]:
image = 's3://my-cog-server/greenland_vel_mosaic500_2000_2001_vv_v02.1_cog.tif'
da = xr.open_rasterio(image, 
                      chunks=dict(x=1024,y=1024)
                     )

In [ ]:
da = da.where(da !=-1, np.nan ) #for velocity magnitude
#da = da.where(da !=-2e9, np.nan ) # vx

In [ ]:
minx, maxx = da.coords['x'].values[[0,-1]]
maxy, miny = da.coords['y'].values[[0,-1]]
midx = (maxx+minx)/2
midy = (maxy+miny)/2

In [ ]:
# Create a linked profile plot - points extents should match image (left, bottom, right and top)
# #not sure logz is implemented/fixed https://github.com/pyviz/holoviews/issues/2195
title = os.path.basename(image)
img = da.sel(band=1).hvplot.image(rasterize=True).opts(title=title, 
                                                      width=500, height=700,
                                                      #cmap='bwr', # for vx or vy
                                                      #clim=(-3e3, 3e3), 
                                                      cmap='viridis',
                                                      clim=(0,3e3),
                                                      active_tools=['point_draw'])
extent = (minx, miny, maxx, maxy)

initPoints = ([-187528, -167767],[-2366098, -2213674])
points = hv.Points(initPoints, extents=extent).opts(size=12) #geoviews requires CRS (if we want a tiled basemap)
table = hv.Table(points, ['x', 'y']).opts(editable=True)
DataLink(points, table)
point_stream = PointDraw(source=points, data=points.columns(), num_objects=2)

@pn.depends(point_stream.param.data)
def plotProfile(data, ds=1000):
    gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(data['x'], data['y']), 
                       crs={'init' :'epsg:3413'}) # Double check EPSG code
    
    gf = gpd.GeoDataFrame(geometry=[LineString(gdf.geometry)], 
                      crs={'init' :'epsg:3413'})
    sgeom = gf.geometry.values[0]
    
    distances = np.arange(0, gf.geometry.length.values[0]+ds, ds) 
    points = [sgeom.interpolate(x) for x in distances]
    xs = xr.DataArray([p.x for p in points], dims='distance', coords=[distances])
    ys = xr.DataArray([p.y for p in points], dims='distance', coords=[distances])
    das = da.sel(band=1, x=xs, y=ys, method='nearest')
    df = das.drop('band').to_dataframe(name='velocity')
    #df = get_profile(point_stream, ds=1000)
    profile = df.hvplot.scatter(y='velocity', logy=True).opts(title=title,
                                                              xlabel='Distance (m)',
                                                              ylabel='Velocity magnitude (m/yr)',
                                                              width=500, height=300)
    
    return profile

app = pn.Row(img*points, pn.Column(table, plotProfile))

In [ ]:
app.servable();